<a href="https://colab.research.google.com/github/NezihAkin/haystack_trial/blob/main/qa_with_haystack_colab_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Test if GPU is enabled
%%bash

nvidia-smi

Fri Apr 28 13:03:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install Haystack
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.1/713.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.9/299.9 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 90.0 MB/s eta 0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [4]:
# Initiate the FAISS Document Store
from haystack.document_stores import FAISSDocumentStore

document_store = FAISSDocumentStore(embedding_dim= 1536, faiss_index_factory_str="Flat", return_embedding=True)


INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).


## Create the Dataset

In [8]:
import pandas as pd

In [12]:
df = pd.read_csv("movies.csv")
df.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,76600,Avatar: The Way of Water,Science Fiction-Adventure-Action,en,Set more than a decade after the events of the first film learn the story of...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,192.0,Released,Return to Pandora.,7.751,6748.0,Sam Worthington-Zoe Saldaña-Sigourney Weaver-Stephen Lang-Kate Winslet-Cliff...,loss of loved one-dying and death-alien life-form-resurrection-sequel-dysfun...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874764-613200-315162-965839-10138...
1,502356,The Super Mario Bros. Movie,Animation-Adventure-Family-Fantasy-Comedy,en,While working underground to fix a water main Brooklyn plumbers—and brothers...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,92.0,Released,NaN,7.556,332.0,Chris Pratt-Anya Taylor-Joy-Charlie Day-Jack Black-Keegan-Michael Key-Seth R...,video game-plumber-magic mushroom-based on video game-aftercreditsstinger-du...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN
2,640146,Ant-Man and the Wasp: Quantumania,Action-Adventure-Science Fiction,en,Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's pare...,4704.903,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,4.732379e+08,125.0,Released,Witness the beginning of a new dynasty.,6.448,1547.0,Paul Rudd-Evangeline Lilly-Jonathan Majors-Kathryn Newton-Michelle Pfeiffer-...,hero-ant-sequel-superhero-based on comic-family-superhero team-aftercreditss...,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,965839-734048-267805-1035806-823999-842942-772515-1058949-1077280-677179-758...
3,677179,Creed III,Drama-Action,en,After dominating the boxing world Adonis Creed has been thriving in both his...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Productions-Outlier Society Produ...,2023-03-01,75000000.0,2.690000e+08,116.0,Released,You can't run from your past.,7.262,1129.0,Michael B. Jordan-Tessa Thompson-Jonathan Majors-Wood Harris-Phylicia Rashād...,philadelphia pennsylvania-husband wife relationship-deaf-sports-sequel-orpha...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-1077280-1058949-772515-937278-640...
4,631842,Knock at the Cabin,Horror-Mystery-Thriller,en,While vacationing at a remote cabin a young girl and her two fathers are tak...,3422.537,Blinding Edge Pictures-Universal Pictures-FilmNation Entertainment-Wishmore-...,2023-02-01,20000000.0,5.200000e+07,100.0,Released,Save your family or save humanity. Make the choice.,6.457,888.0,Dave Bautista-Jonathan Groff-Ben Aldridge-Kristen Cui-Nikki Amuka-Bird-Ruper...,based on novel or book-sacrifice-cabin-faith-end of the world-apocalypse-hom...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-1048522-785084-1058617-986054-640...


In [13]:
df.shape

(722924, 20)

In [14]:
df['release_date'] = pd.to_datetime(df['release_date'])
df.dtypes

id                               int64
title                           object
genres                          object
original_language               object
overview                        object
popularity                     float64
production_companies            object
release_date            datetime64[ns]
budget                         float64
revenue                        float64
runtime                        float64
status                          object
tagline                         object
vote_average                   float64
vote_count                     float64
credits                         object
keywords                        object
poster_path                     object
backdrop_path                   object
recommendations                 object
dtype: object

In [15]:
# I filtered the momvies after Jan 2022. Because, end date of ChatGPT is Sep 2021. Also, I only picked movies Released after that day.
df_filtered = df[df['release_date']>='2022-01-01']
df_filtered = df_filtered[df_filtered['status']=='Released']
df_filtered.shape

(12573, 20)

In [16]:
df_filtered.release_date.min()

Timestamp('2022-01-01 00:00:00')

In [18]:
# Some preprocessing
df_filtered['credits'] = df_filtered['credits'].str.replace("-",", ")
df_filtered['genres'] = df_filtered['genres'].str.replace("-",", ")
df_filtered.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,76600,Avatar: The Way of Water,"Science Fiction, Adventure, Action",en,Set more than a decade after the events of the first film learn the story of...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,192.0,Released,Return to Pandora.,7.751,6748.0,"Sam Worthington, Zoe Saldaña, Sigourney Weaver, Stephen Lang, Kate Winslet, ...",loss of loved one-dying and death-alien life-form-resurrection-sequel-dysfun...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874764-613200-315162-965839-10138...
1,502356,The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",en,While working underground to fix a water main Brooklyn plumbers—and brothers...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,92.0,Released,NaN,7.556,332.0,"Chris Pratt, Anya Taylor, Joy, Charlie Day, Jack Black, Keegan, Michael Key,...",video game-plumber-magic mushroom-based on video game-aftercreditsstinger-du...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN
2,640146,Ant-Man and the Wasp: Quantumania,"Action, Adventure, Science Fiction",en,Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's pare...,4704.903,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,4.732379e+08,125.0,Released,Witness the beginning of a new dynasty.,6.448,1547.0,"Paul Rudd, Evangeline Lilly, Jonathan Majors, Kathryn Newton, Michelle Pfeif...",hero-ant-sequel-superhero-based on comic-family-superhero team-aftercreditss...,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,965839-734048-267805-1035806-823999-842942-772515-1058949-1077280-677179-758...
3,677179,Creed III,"Drama, Action",en,After dominating the boxing world Adonis Creed has been thriving in both his...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Productions-Outlier Society Produ...,2023-03-01,75000000.0,2.690000e+08,116.0,Released,You can't run from your past.,7.262,1129.0,"Michael B. Jordan, Tessa Thompson, Jonathan Majors, Wood Harris, Phylicia Ra...",philadelphia pennsylvania-husband wife relationship-deaf-sports-sequel-orpha...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-1077280-1058949-772515-937278-640...
4,631842,Knock at the Cabin,"Horror, Mystery, Thriller",en,While vacationing at a remote cabin a young girl and her two fathers are tak...,3422.537,Blinding Edge Pictures-Universal Pictures-FilmNation Entertainment-Wishmore-...,2023-02-01,20000000.0,5.200000e+07,100.0,Released,Save your family or save humanity. Make the choice.,6.457,888.0,"Dave Bautista, Jonathan Groff, Ben Aldridge, Kristen Cui, Nikki Amuka, Bird,...",based on novel or book-sacrifice-cabin-faith-end of the world-apocalypse-hom...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-1048522-785084-1058617-986054-640...


In [19]:
df_filtered.loc[2,'overview']

"Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's parents Janet van Dyne and Hank Pym and Scott's daughter Cassie Lang find themselves exploring the Quantum Realm interacting with strange new creatures and embarking on an adventure that will push them beyond the limits of what they thought possible."

In [20]:
# Create a context for the prompt from the dataset
df_filtered['context'] = df_filtered.apply(lambda row: f"The title of the movie is {row['title']}. The genre of this movie is {row['genres']}. This movie is released on {row['release_date']}. The budget of the movie is ${row['budget']/1000000} million and total revenue of the movie is ${row['revenue']/1000000} million. The plot of the movie is {row['overview']}. Average vote of the movie is {row['vote_average']} out of 10 in {row['vote_count']} votes.", axis =1) 

In [21]:
df_filtered.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations,context
0,76600,Avatar: The Way of Water,"Science Fiction, Adventure, Action",en,Set more than a decade after the events of the first film learn the story of...,9366.788,20th Century Studios-Lightstorm Entertainment,2022-12-14,350000000.0,2.312336e+09,...,Released,Return to Pandora.,7.751,6748.0,"Sam Worthington, Zoe Saldaña, Sigourney Weaver, Stephen Lang, Kate Winslet, ...",loss of loved one-dying and death-alien life-form-resurrection-sequel-dysfun...,/t6HIqrRAclMCA60NsSmeqe9RmNV.jpg,/s16H6tpK2utvwDtzZ8Qy4qm5Emw.jpg,183392-111332-702432-505642-1064215-436270-874764-613200-315162-965839-10138...,The title of the movie is Avatar: The Way of Water. The genre of this movie ...
1,502356,The Super Mario Bros. Movie,"Animation, Adventure, Family, Fantasy, Comedy",en,While working underground to fix a water main Brooklyn plumbers—and brothers...,5132.098,Universal Pictures-Illumination-Nintendo,2023-04-05,100000000.0,5.800000e+07,...,Released,NaN,7.556,332.0,"Chris Pratt, Anya Taylor, Joy, Charlie Day, Jack Black, Keegan, Michael Key,...",video game-plumber-magic mushroom-based on video game-aftercreditsstinger-du...,/qNBAXBIQlnOThrVvA6mA2B5ggV6.jpg,/iw0Na1UBHgA5BgifwmQ8vKhlWgA.jpg,NaN,The title of the movie is The Super Mario Bros. Movie. The genre of this mov...
2,640146,Ant-Man and the Wasp: Quantumania,"Action, Adventure, Science Fiction",en,Super-Hero partners Scott Lang and Hope van Dyne along with with Hope's pare...,4704.903,Marvel Studios-Kevin Feige Productions,2023-02-15,200000000.0,4.732379e+08,...,Released,Witness the beginning of a new dynasty.,6.448,1547.0,"Paul Rudd, Evangeline Lilly, Jonathan Majors, Kathryn Newton, Michelle Pfeif...",hero-ant-sequel-superhero-based on comic-family-superhero team-aftercreditss...,/ngl2FKBlU4fhbdsrtdom9LVLBXw.jpg,/3CxUndGhUcZdt1Zggjdb2HkLLQX.jpg,965839-734048-267805-1035806-823999-842942-772515-1058949-1077280-677179-758...,The title of the movie is Ant-Man and the Wasp: Quantumania. The genre of th...
3,677179,Creed III,"Drama, Action",en,After dominating the boxing world Adonis Creed has been thriving in both his...,3994.342,Metro-Goldwyn-Mayer-Proximity Media-Balboa Productions-Outlier Society Produ...,2023-03-01,75000000.0,2.690000e+08,...,Released,You can't run from your past.,7.262,1129.0,"Michael B. Jordan, Tessa Thompson, Jonathan Majors, Wood Harris, Phylicia Ra...",philadelphia pennsylvania-husband wife relationship-deaf-sports-sequel-orpha...,/cvsXj3I9Q2iyyIo95AecSd1tad7.jpg,/5i6SjyDbDWqyun8klUuCxrlFbyw.jpg,965839-267805-943822-842942-1035806-823999-1077280-1058949-772515-937278-640...,"The title of the movie is Creed III. The genre of this movie is Drama, Actio..."
4,631842,Knock at the Cabin,"Horror, Mystery, Thriller",en,While vacationing at a remote cabin a young girl and her two fathers are tak...,3422.537,Blinding Edge Pictures-Universal Pictures-FilmNation Entertainment-Wishmore-...,2023-02-01,20000000.0,5.200000e+07,...,Released,Save your family or save humanity. Make the choice.,6.457,888.0,"Dave Bautista, Jonathan Groff, Ben Aldridge, Kristen Cui, Nikki Amuka, Bird,...",based on novel or book-sacrifice-cabin-faith-end of the world-apocalypse-hom...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-1048522-785084-1058617-986054-640...,The title of the movie is Knock at the Cabin. The genre of this movie is Hor...


In [23]:
df_filtered.loc[0, 'context']

'The title of the movie is Avatar: The Way of Water. The genre of this movie is Science Fiction, Adventure, Action. This movie is released on 2022-12-14 00:00:00. The budget of the movie is $350.0 million and total revenue of the movie is $2312.335665 million. The plot of the movie is Set more than a decade after the events of the first film learn the story of the Sully family (Jake Neytiri and their kids) the trouble that follows them the lengths they go to keep each other safe the battles they fight to stay alive and the tragedies they endure.. Average vote of the movie is 7.751 out of 10 in 6748.0 votes.'

In [24]:
# Create the Document Store
document_store = FAISSDocumentStore(embedding_dim=1536,
                                    faiss_index_factory_str="Flat",
                                    return_embedding=True)

In [25]:
document_store

In [26]:
from haystack import Document

In [27]:
# Put the information inside of documents list to embed document store. 
# We can think of this as our documents
titles = list(df_filtered['title'].values)
contexts = list(df_filtered['context'].values)

documents = []

for title, context in zip(titles, contexts):
  documents.append(Document(content=context, meta={"name":title or ""}))

In [29]:
# Delete existing documents inside of document store if there are any
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

Writing Documents:   0%|          | 0/12573 [00:00<?, ?it/s]

In [31]:
import tiktoken

def num_tokens_from_string(string, encoding_name):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [33]:
# Calculate number of tokens 
df_filtered['token_count'] = df_filtered['context'].apply(lambda text: num_tokens_from_string(text,'cl100k_base'))
df_filtered['word_count'] = df_filtered['context'].apply(lambda text: len(text.split()))

In [35]:
print(df_filtered.token_count.sum())
print(df_filtered.word_count.sum())

1764302
1263523


In [36]:
df_filtered.token_count.sum() * 0.0004 / 1000
# I will pay 70 cents for these embeddings

0.7057208

In [45]:
import os
openai_api_key = os.getenv('OPENAI_API_KEY')

In [46]:
# Create embeddings with OPENAI model
from haystack.nodes.retriever import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    batch_size = 8,
    embedding_model = 'text-embedding-ada-002',
    api_key = openai_api_key,
    max_seq_len= 8191
)

document_store.update_embeddings(retriever)
# This took around 11 mins.

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model text-embedding-ada-002
INFO:haystack.document_stores.faiss:Updating embeddings for 11708 docs...


Updating Embedding:   0%|          | 0/11708 [00:00<?, ? docs/s]

Calculating embeddings:   0%|          | 0/1250 [00:00<?, ?it/s]

Calculating embeddings:   0%|          | 0/214 [00:00<?, ?it/s]

In [49]:
# Save the document store 
document_store.save('my_example_faiss_index.faiss')

In [59]:
from haystack.nodes import OpenAIAnswerGenerator
# Create the generator from OpenAI
generator = OpenAIAnswerGenerator(
    api_key = openai_api_key,
    model = 'text-davinci-003',
    max_tokens = 128,
    top_k = 2
)

In [60]:
from haystack.pipelines import GenerativeQAPipeline
# Initiate the pipeline
pipeline = GenerativeQAPipeline(generator = generator, retriever = retriever)

In [61]:
# Time to get the answers

query = "What is the plot of the movie Knock at the Cabin and what was the revenue generated from this movie?"


Answer of ChatGPT is as follows:

I'm sorry, but there is no record of a movie called "Knock at the Cabin." It's possible that the movie was released under a different title or that it was a low-budget independent film that did not receive wide distribution or generate significant revenue. If you have any additional information or context about the movie, I may be able to assist you further.

As it can be seen, it doesn't know about the movie.

In [63]:
response = pipeline.run(query = query)

Calculating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [77]:
response['answers'][0].answer

' The plot of the movie Knock at the Cabin is While vacationing at a remote cabin a young girl and her two fathers are taken hostage by four armed strangers who demand that the family make an unthinkable choice to avert the apocalypse. With limited access to the outside world the family must decide what they believe before all is lost. The total revenue generated from this movie is $52.0 million.'

In [78]:
df_filtered[df_filtered['title']=='Knock at the Cabin']

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,...,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations,context,token_count,word_count
4,631842,Knock at the Cabin,"Horror, Mystery, Thriller",en,While vacationing at a remote cabin a young girl and her two fathers are tak...,3422.537,Blinding Edge Pictures-Universal Pictures-FilmNation Entertainment-Wishmore-...,2023-02-01,20000000.0,52000000.0,...,6.457,888.0,"Dave Bautista, Jonathan Groff, Ben Aldridge, Kristen Cui, Nikki Amuka, Bird,...",based on novel or book-sacrifice-cabin-faith-end of the world-apocalypse-hom...,/dm06L9pxDOL9jNSK4Cb6y139rrG.jpg,/zWDMQX0sPaW2u0N2pJaYA8bVVaJ.jpg,1058949-646389-772515-505642-143970-667216-1048522-785084-1058617-986054-640...,The title of the movie is Knock at the Cabin. The genre of this movie is Hor...,149,113


In [ ]:
# As it can be seen answer is there!!